In [1]:
import os
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import re

In [37]:
# TODO: add case type to criminal case reports in PDCMS
NAMES = ["Case#", "Open Date", "Closed Date", "Court", "Latest Event",
             "Intake Type", "TC C/L", "Top Charge", "Dispo", "Case Type"]
CASE_TYPES = CategoricalDtype(categories=["Homicide",
                                          "Violent Felony",
                                          "Non-Violent Felony",
                                          "Misdemeanors, Violation, or Other"],
                              ordered=True)

In [36]:
def read_and_concat_csvs(csv_names = False):
    """Reads all CSVs given, gives them cleaned column names from NAMES,
    then combines them vertically.
    
    arguments:
    csv_names        list of file names as strings
    """
    return pd.concat(
            [pd.read_csv(fname, names=NAMES, header=0) for fname in csv_names]
            ).reset_index(drop=True)

In [4]:
def classify_tc(tc):
    """Classifies top charge class and levels into MOCJ-requested categories.
    """
    tc = str(tc) #coerce TC type to string
    if tc == "H":
        return ("Homicide")
    elif ("FL" in tc) or ("FV" in tc):
        return ("Violent Felony")
    elif (tc.endswith("F") or tc == "F"):
        return ("Non-Violent Felony")
    else:
        return ("Misdemeanor, Violation, or Other")

## Opened Cases
Criminal, VOP, and VOCD cases opened in the reporting year

In [ ]:
cases_opened = read_and_concat_csvs(["U:\Data and Analytics\Projects\ILS\crim_cases_opened.csv",
                                     "U:\Data and Analytics\Projects\ILS\post_dispo_cases_closed.csv"])
cases_opened["tc_type"] = case_opened["TC C/L"].apply(lambda x: classify_tc(tc))

## Closed Cases
Criminal, VOCD, and VOP cases closed in the reporting year.

In [ ]:
crim_cases_closed = read_and_concat_csvs(["U:\Data and Analytics\Projects\ILS\crim_cases_closed.csv",
                                          "U:\"U:\Data and Analytics\Projects\ILS\post_dispo_cases_closed.csv"])